Making prediction on 'Test.csv' data file


In [1]:
#importing the model from saved file
import joblib

req_model=joblib.load("randomforest_adview.pkl")


In [2]:
req_model

RandomForestRegressor()

In [3]:
#Loading the data file for prediction
import pandas as pd
test_dataset=pd.read_csv("test.csv")


In [4]:
test_dataset.head()

,vidid,views,likes,dislikes,comment,published,duration,category
0,VID_1054,440238,6153,218,1377,2017-02-18,PT7M29S,B
1,VID_18629,1040132,8171,340,1047,2016-06-28,PT6M29S,F
2,VID_13967,28534,31,11,1,2014-03-10,PT37M54S,D
3,VID_19442,1316715,2284,250,274,2010-06-05,PT9M55S,G
4,VID_770,1893173,2519,225,116,2016-09-03,PT3M8S,B


In [5]:
test_dataset.shape

(8764, 8)

In [6]:
test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8764 entries, 0 to 8763
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   vidid      8764 non-null   object
 1   views      8764 non-null   object
 2   likes      8764 non-null   object
 3   dislikes   8764 non-null   object
 4   comment    8764 non-null   object
 5   published  8764 non-null   object
 6   duration   8764 non-null   object
 7   category   8764 non-null   object
dtypes: object(8)
memory usage: 547.9+ KB


In [7]:
(test_dataset['comment']=='F').sum()

151

Data Preprocessing

In [8]:
#changing datatype of some columns from object to int

import numpy as np

test_dataset['views']=[int(i) if i.isnumeric() else np.NaN for i in test_dataset['views'] ]
test_dataset['likes']=pd.Series([int(i) if i.isnumeric() else np.NaN for i in test_dataset['likes'] ])
test_dataset['dislikes']=[int(i) if i.isnumeric() else np.NaN for i in test_dataset['dislikes'] ]
test_dataset['comment']=[int(i) if i.isnumeric() else np.NaN for i in test_dataset['comment'] ]

In [9]:
test_dataset.dropna(inplace=True)
test_dataset.shape

(8549, 8)

In [10]:
test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8549 entries, 0 to 8763
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   vidid      8549 non-null   object 
 1   views      8549 non-null   float64
 2   likes      8549 non-null   float64
 3   dislikes   8549 non-null   float64
 4   comment    8549 non-null   float64
 5   published  8549 non-null   object 
 6   duration   8549 non-null   object 
 7   category   8549 non-null   object 
dtypes: float64(4), object(4)
memory usage: 601.1+ KB


In [11]:
#removing unnecessary column
test_dataset.drop(columns="vidid",inplace=True)

In [12]:
# Label encoding non integer data

from sklearn.preprocessing import LabelEncoder

encoder=LabelEncoder()

test_dataset['category']= encoder.fit_transform(test_dataset['category'])
test_dataset['published']= encoder.fit_transform(test_dataset['published'])


In [13]:
#Cleaning the "duration" column of data

def clean_dur(str_duration):
    str_duration.upper().strip()
    str_duration=str_duration[2:]
    newstr_dur=""
    
    if "H" in str_duration:
        for i in range(len(str_duration)):
            if str_duration[i]=='H':
                hour_mark=i
                word=str_duration[:i]
                if word.isnumeric==False:
                    word="0"
    else:
        hour_mark=0
        word="0"
    newstr_dur+=word+":"
    
    if "M" in str_duration:
        for i in range(len(str_duration)):
            if str_duration[i]=='M':
                if hour_mark==0:
                    word=str_duration[:i]
                else:
                    word=str_duration[hour_mark+1:i]
                if word.isnumeric==False:
                    word="0"
                min_mark=i
    else:
        min_mark=0
        word="0"
    newstr_dur+=word+":"

    if "S" in str_duration:
        for i in range(len(str_duration)):
            if str_duration[i]=='S':
                if hour_mark==0 and min_mark==0:
                    word=str_duration[:i]
                elif min_mark==0:
                    word=str_duration[hour_mark+1:i]
                else:
                    word=str_duration[min_mark+1:i]
                if word=="P":
                    word="0"
    else:
        word="0"
    newstr_dur+=word

    return newstr_dur

def format_dur(new_dur):
    new_dur=new_dur.split(":")
    ans=(3600*int(new_dur[0]))+(60*int(new_dur[1]))+(int(new_dur[2]))
    return ans

In [14]:
#using function to clean data
test_dataset['duration']=test_dataset['duration'].apply(clean_dur)
test_dataset['duration']=test_dataset['duration'].apply(format_dur)

In [15]:
test_dataset.head()

,views,likes,dislikes,comment,published,duration,category
0,440238.0,6153.0,218.0,1377.0,2053,449,1
1,1040132.0,8171.0,340.0,1047.0,1825,389,5
2,28534.0,31.0,11.0,1.0,1009,2274,3
3,1316715.0,2284.0,250.0,274.0,116,595,6
4,1893173.0,2519.0,225.0,116.0,1892,188,1


In [16]:
#Scaling data
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

test_dataset=scaler.fit_transform(test_dataset)

In [17]:
test_dataset

array([[1.95520958e-03, 1.92237996e-02, 6.05168920e-03, ...,
        9.75296912e-01, 1.07451417e-02, 1.42857143e-01],
       [4.61965571e-03, 2.55286311e-02, 9.43841435e-03, ...,
        8.66983373e-01, 9.29309552e-03, 7.14285714e-01],
       [1.26614645e-04, 9.68532080e-05, 3.05360464e-04, ...,
        4.79334917e-01, 5.49115462e-02, 4.28571429e-01],
       ...,
       [8.40188347e-03, 1.43905121e-02, 3.07858868e-02, ...,
        8.66033254e-01, 1.03821302e-02, 7.14285714e-01],
       [3.17494650e-03, 5.12072284e-03, 4.19176637e-03, ...,
        8.20902613e-01, 4.59814622e-03, 4.28571429e-01],
       [2.34068537e-05, 3.74915644e-05, 2.77600422e-05, ...,
        9.48218527e-01, 1.47382687e-02, 4.28571429e-01]])

Making Prediction

In [18]:
test_dataset.shape

(8549, 7)

In [19]:
test_pred=req_model.predict(test_dataset)
test_pred

array([522.41, 549.11,  90.22, ..., 113.41,   3.78,  39.3 ])

In [25]:
test_pred=pd.DataFrame(data={"Predicted Adview": pd.Series(test_pred)})

In [27]:
test_pred.to_csv("PredictedAdview.csv")